In [120]:
import yfinance as yf
import pandas as pd
import numpy as np
import streamlit as st

In [104]:
TICKER_SYMBOL = "RELIANCE.NS"
PEER_TICKERS = ["TCS.NS", "HDFCBANK.NS", "ICICIBANK.NS"]

In [48]:
ticker = yf.Ticker(TICKER_SYMBOL) #get_ticker_object
current_price = ticker.info.get('currentPrice') #get_realtime_price
#yf.download('RELIANCE.NS', period="1d", interval="1m", progress=False)['Close']

In [49]:
# calculate DCF
current_price = ticker.info.get('currentPrice')
eps_ttm = ticker.info.get('trailingEps')
growth_rate = ticker.info.get('earningsGrowth')
discount_rate = 0.12 # Standard WACC assumption 
terminal_multiple = 15
# Simplified 5-year DCF projection (using EPS for proxy)
projected_eps = [eps_ttm * (1 + growth_rate)**i for i in range(1, 6)]

# Calculate Present Value (PV)
pv_cash_flows = [eps / (1 + discount_rate)**i for i, eps in enumerate(projected_eps, 1)]
terminal_value = projected_eps[-1] * terminal_multiple
pv_terminal_value = terminal_value / (1 + discount_rate)**5
intrinsic_value = round(sum(pv_cash_flows) + pv_terminal_value, 2)
margin_of_safety = round(100 * (intrinsic_value - current_price) / intrinsic_value, 2)


print(f"""
      Current Price : {current_price}, 
      TTM EPS : {eps_ttm}, 
      Growth Rate : {growth_rate} 
      Projected EPS : {projected_eps}
      PV Cash Flow : {pv_cash_flows}
      Terminal Value : {terminal_value}
      PV Terminal Value : {pv_terminal_value}
      Intrinsic Value : {intrinsic_value}
      Margin Of Safety : {margin_of_safety}
      """)

{
    'current_price': current_price,
    'intrinsic_value': intrinsic_value,
    'margin_of_safety': margin_of_safety,
    'inputs': {
        'EPS (TTM)': f"₹{eps_ttm:.2f}", 
        'LT Growth': f"{growth_rate * 100:.1f}%",
        'Discount Rate': f"{discount_rate * 100:.0f}%"
    }
}



      Current Price : 1375.0, 
      TTM EPS : 60.24, 
      Growth Rate : 0.784 
      Projected EPS : [107.46816000000001, 191.72319744, 342.03418423296, 610.1889846716007, 1088.5771486541357]
      PV Cash Flow : [95.95371428571428, 152.84055918367346, 243.4531764139941, 387.7861310022906, 617.6879086679344]
      Terminal Value : 16328.657229812035
      PV Terminal Value : 9265.318630019015
      Intrinsic Value : 10763.04
      Margin Of Safety : 87.22
      


{'current_price': 1375.0,
 'intrinsic_value': 10763.04,
 'margin_of_safety': 87.22,
 'inputs': {'EPS (TTM)': '₹60.24',
  'LT Growth': '78.4%',
  'Discount Rate': '12%'}}

In [101]:
# Technical Data

df_multi = yf.download(
        TICKER_SYMBOL, 
        period="5y", # <-- Set period to 5 years (recommended for 200-day SMA)
        interval="1d",
        progress=False,
    )
df_multi.columns = df_multi.columns.get_level_values(0)

# 3. (Optional but recommended) Rename the 'Adj Close' column to simply 'Close' 
# if you prefer a cleaner column name for the adjusted closing price
if 'Adj Close' in df_multi.columns:
    df_multi = df_multi.rename(columns={'Adj Close': 'Close'})

# The resulting DataFrame now has a single-level index showing only price metrics
df = df_multi

df['SMA_50'] = df['Close'].rolling(window=50).mean().round(2)
df['SMA_200'] = df['Close'].rolling(window=200).mean().round(2)

delta = df['Close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()

rs = gain / loss
df['RSI'] = 100 - (100 / (1 + rs)).round(2)
df = df.dropna(subset=['SMA_50', 'SMA_200', 'RSI'])
df

/var/folders/4y/rp2s6cx97jbglhhwgx2b4cth0000gn/T/ipykernel_2560/1505715264.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_multi = yf.download(


Price,Close,High,Low,Open,Volume,SMA_50,SMA_200,RSI
Date,,,,,,,,
2021-08-02,943.012512,946.152073,928.747886,934.731298,10925104,966.25,923.12,47.79
2021-08-03,949.951355,951.430147,939.235856,950.975148,11817299,967.24,922.70,47.72
2021-08-04,957.231567,961.895414,949.610075,952.977166,13169800,968.58,922.30,53.35
2021-08-05,971.109375,980.459815,957.458979,962.031838,21252078,970.14,922.16,58.90
2021-08-06,950.542908,976.888089,945.856254,969.152862,16620987,971.23,921.97,46.20
...,...,...,...,...,...,...,...,...
2025-10-07,1384.800049,1395.900024,1375.900024,1376.000000,11244040,1385.68,1340.91,34.57
2025-10-08,1367.400024,1389.000000,1364.500000,1384.800049,8525081,1384.80,1341.74,28.40
2025-10-09,1377.800049,1380.000000,1369.099976,1374.000000,8826633,1384.27,1342.54,36.90


In [116]:
# valuation_and_peer_data

company_data = {
        'P/E (TTM)': ticker.info.get('trailingPE', np.nan),
        'P/B': ticker.info.get('priceToBook', np.nan),
        'EV/EBITDA': ticker.info.get('enterpriseToEbitda', np.nan),
        '5Y Avg P/E': ticker.info.get('fiftyDayAverage', np.nan) / ticker.info.get('trailingEps', 1) # Crude proxy, not actual 5Y avg
    }

peer_metrics = []
multi_ticker_data = yf.download(PEER_TICKERS, period="1d", progress=False)

for peer_ticker in PEER_TICKERS:
        try:
            peer_info = yf.Ticker(peer_ticker).info
            peer_metrics.append({
                'Stock': peer_ticker,
                'P/E (TTM)': peer_info.get('trailingPE', np.nan),
                'P/B': peer_info.get('priceToBook', np.nan),
                'EV/EBITDA': peer_info.get('enterpriseToEbitda', np.nan),
            })
        except:
            peer_metrics.append({'Stock': peer_ticker, 'P/E (TTM)': np.nan, 'P/B': np.nan, 'EV/EBITDA': np.nan})

quality_ratios = {
        'ROE (%)': ticker.info.get('returnOnEquity', np.nan) * 100,
        'Debt/Equity': ticker.info.get('debtToEquity', np.nan) / 100,
        'Current Ratio': ticker.info.get('currentRatio', np.nan),
        'Interest Coverage': ticker.info.get('ebitda') / ticker.info.get('interestExpense') if ticker.info.get('interestExpense') else np.nan,
    }

{
        'company_valuation': {k: round(v, 2) if isinstance(v, (int, float)) and not np.isnan(v) else v for k, v in company_data.items()},
        'peer_comparison': peer_metrics,
        'quality_ratios': {k: round(v, 2) if isinstance(v, (int, float)) and not np.isnan(v) else v for k, v in quality_ratios.items()}
    }

/var/folders/4y/rp2s6cx97jbglhhwgx2b4cth0000gn/T/ipykernel_2560/3116742934.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  multi_ticker_data = yf.download(PEER_TICKERS, period="1d", progress=False)


{'company_valuation': {'P/E (TTM)': 22.83,
  'P/B': 2.21,
  'EV/EBITDA': 12.74,
  '5Y Avg P/E': 22.99},
 'peer_comparison': [{'Stock': 'TCS.NS',
   'P/E (TTM)': 22.017864,
   'P/B': 10.225649,
   'EV/EBITDA': 15.727},
  {'Stock': 'HDFCBANK.NS',
   'P/E (TTM)': 22.331429,
   'P/B': 2.7576046,
   'EV/EBITDA': nan},
  {'Stock': 'ICICIBANK.NS',
   'P/E (TTM)': 18.74949,
   'P/B': 2.9865806,
   'EV/EBITDA': nan}],
 'quality_ratios': {'ROE (%)': nan,
  'Debt/Equity': 0.37,
  'Current Ratio': nan,
  'Interest Coverage': nan}}

In [125]:
import altair as alt

In [129]:
tech_df = df

current_close = tech_df['Close'].iloc[-1].item()
sma_50 = tech_df['SMA_50'].iloc[-1].item()
sma_200 = tech_df['SMA_200'].iloc[-1].item()

PRIMARY_COLOR = "#00B0FF" 
SECONDARY_COLOR = "#FF9900"

tech_df

Price,Close,High,Low,Open,Volume,SMA_50,SMA_200,RSI
Date,,,,,,,,
2021-08-02,943.012512,946.152073,928.747886,934.731298,10925104,966.25,923.12,47.79
2021-08-03,949.951355,951.430147,939.235856,950.975148,11817299,967.24,922.70,47.72
2021-08-04,957.231567,961.895414,949.610075,952.977166,13169800,968.58,922.30,53.35
2021-08-05,971.109375,980.459815,957.458979,962.031838,21252078,970.14,922.16,58.90
2021-08-06,950.542908,976.888089,945.856254,969.152862,16620987,971.23,921.97,46.20
...,...,...,...,...,...,...,...,...
2025-10-07,1384.800049,1395.900024,1375.900024,1376.000000,11244040,1385.68,1340.91,34.57
2025-10-08,1367.400024,1389.000000,1364.500000,1384.800049,8525081,1384.80,1341.74,28.40
2025-10-09,1377.800049,1380.000000,1369.099976,1374.000000,8826633,1384.27,1342.54,36.90


In [149]:
tech_df.columns.name = None
tech_df.columns

Index(['Close', 'High', 'Low', 'Open', 'Volume', 'SMA_50', 'SMA_200', 'RSI'], dtype='object')

In [150]:
tech_df.columns.name = None

price_chart = alt.Chart(tech_df.reset_index()).transform_fold(
        ['Close', 'SMA_50', 'SMA_200'],
        as_=['Metric', 'Value']
    ).mark_line().encode(
        x=alt.X('Date:T', title="Date"),
        y=alt.Y('Value:Q', title="Price (₹)"),
        color=alt.Color('Metric:N', scale=alt.Scale(range=['black', PRIMARY_COLOR, SECONDARY_COLOR])),
        tooltip=['Date:T', 'Metric:N', alt.Tooltip('Value:Q', format=".2f")]
    ).properties(height=400)

price_chart

alt.Chart(...)